In [1]:
import os
os.chdir("../../../")

import pandas as pd
import numpy as np

import re
import nltk
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel 
import spacy

from scripts.python.nlp.utils import *
from scripts.python.nlp.lda import *

In [2]:
target = os.getcwd() + "/data/text/solomon_islands/"
news_path = [filename for filename in os.listdir(target)  if "news" in filename]

In [3]:
news = pd.DataFrame()
for path in news_path:
    temp = pd.read_csv(target + path).drop("Unnamed: 0", axis=1)
    temp["from"] = path.replace("_news.csv", "")
    if news.empty:
        news = temp
    else: 
        news = pd.concat([news, temp], axis=0)

In [4]:
news = news[news.news.isna() != True].reset_index(drop=True)

In [5]:
en = spacy.load('en_core_web_sm')
spacy_stopwords = list(en.Defaults.stop_words)

In [6]:
nltk_stopwords = stopwords.words("english")
nltk_unique = [sw for sw in nltk_stopwords if sw not in spacy_stopwords]
spacy_stopwords.extend(nltk_unique)

In [7]:
data = news.news.values.tolist()
data_words = list(sent_to_words(data))

In [8]:
# nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp = spacy.load("en_core_web_sm")

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)

trigram = gensim.models.Phrases(bigram[data_words], min_count=5, threshold=100)
trigram_mod = gensim.models.phrases.Phraser(trigram)

texts_preprocessed = preprocess_text(data_words, spacy_stopwords, bigram_mod,
                                     trigram_mod, nlp)

Stopwords has been done.


In [9]:
id2word = corpora.Dictionary(texts_preprocessed)
texts = texts_preprocessed
corpus = [id2word.doc2bow(text) for text in texts]

In [11]:
from gensim.models.wrappers import LdaMallet
mallet_path = os.getcwd() + '/../mallet-2.0.8/bin/mallet'
ldamallet = LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

Mallet LDA: 20 topics, 5 topic bits, 11111 topic mask
Data loaded.
max tokens: 2592
total tokens: 4283306
<10> LL/token: -10.3998
<20> LL/token: -9.25882
<30> LL/token: -8.68029
<40> LL/token: -8.48979

0	2.5	country economic business system increase economy percent sector service investment growth provide financial access population improve rate impact level opportunity 
1	2.5	police officer rsipf incident force report investigation vehicle arrest medium suspect man operation station commissioner public information law involve continue 
2	2.5	service health medical provide work support disaster hospital people supply care equipment receive facility staff clinic patient centre include province 
3	2.5	court case charge accuse year allege victim man high prosecution matter public face yesterday allegedly order hear trial time evidence 
4	2.5	government policy national budget development plan ministry year address ensure fund process province report review priority corruption issue consul

<150> LL/token: -8.25721
<160> LL/token: -8.25357
<170> LL/token: -8.25159
<180> LL/token: -8.25008
<190> LL/token: -8.24925

0	2.5	business economic increase year country service percent economy investment growth financial sector high cost access rate expect provide revenue opportunity 
1	2.5	police officer rsipf force report incident investigation vehicle operation arrest medium suspect public station law commissioner continue information involve death 
2	2.5	service health medical provide support hospital disaster care work people equipment supply receive team include clinic patient centre nurse staff 
3	2.5	court case charge accuse year man allege victim prosecution high matter allegedly face public yesterday hear trial order time lawyer 
4	2.5	government policy development national plan ensure budget system address ministry strategy process priority consultation review stakeholder important implementation reform corruption 
5	2.5	issue company land report claim log question concer

<300> LL/token: -8.239
<310> LL/token: -8.23845
<320> LL/token: -8.23922
<330> LL/token: -8.23946
<340> LL/token: -8.23862

0	2.5	business year economic increase country service percent economy investment growth financial sector high cost access expect rate opportunity revenue provide 
1	2.5	police officer rsipf force report incident vehicle investigation operation medium arrest suspect public station law commissioner continue involve information act 
2	2.5	service health medical provide support hospital work care people disaster equipment supply team include receive child clinic patient nurse staff 
3	2.5	court case charge accuse year man allege victim prosecution matter high allegedly face public yesterday hear trial time order lawyer 
4	2.5	government policy development plan national ensure budget system address ministry strategy stakeholder priority consultation process important review implement implementation reform 
5	2.5	issue land company report claim log medium question conce

<450> LL/token: -8.23836
<460> LL/token: -8.23823
<470> LL/token: -8.23719
<480> LL/token: -8.23735
<490> LL/token: -8.23762

0	2.5	business year economic increase country service percent economy investment growth financial sector high cost access rate expect opportunity revenue provide 
1	2.5	police officer rsipf force report investigation vehicle incident operation medium public suspect arrest station law commissioner continue information involve community 
2	2.5	service health medical provide support hospital care work include supply receive equipment people child team clinic patient staff nurse centre 
3	2.5	court case charge accuse year man allege victim prosecution matter high allegedly public face yesterday hear trial order count lawyer 
4	2.5	government policy development plan national ensure budget system address strategy ministry stakeholder priority consultation important process implementation implement review key 
5	2.5	issue land company report claim medium log concern qu

<600> LL/token: -8.23797
<610> LL/token: -8.23895
<620> LL/token: -8.23949
<630> LL/token: -8.23861
<640> LL/token: -8.23917

0	2.5	business economic year increase country percent economy service investment growth financial sector high cost rate access revenue opportunity expect provide 
1	2.5	police officer rsipf force report vehicle investigation operation incident medium public suspect arrest station commissioner law continue information involve act 
2	2.5	service health medical provide support hospital people work care child receive include equipment supply clinic patient staff team nurse centre 
3	2.5	court case charge accuse man year allege victim prosecution matter high allegedly face public yesterday hear trial time order lawyer 
4	2.5	government development policy plan national ensure budget system address strategy priority stakeholder ministry important sector key consultation implement implementation process 
5	2.5	issue land company report medium claim log concern question 

<750> LL/token: -8.23958
<760> LL/token: -8.24139
<770> LL/token: -8.241
<780> LL/token: -8.24088
<790> LL/token: -8.24017

0	2.5	business year increase economic country percent economy service investment growth financial sector cost high rate expect total revenue government access 
1	2.5	police officer rsipf force report vehicle investigation operation incident medium suspect public arrest station law commissioner continue involve information community 
2	2.5	service health medical provide support hospital work care people child include receive equipment supply clinic patient staff nurse team centre 
3	2.5	court case charge accuse man year allege victim prosecution matter high allegedly face public hear yesterday trial order lawyer count 
4	2.5	government development policy national plan ensure system address strategy budget stakeholder sector priority ministry important key consultation implementation process implement 
5	2.5	issue land company medium claim log concern question deal 

<900> LL/token: -8.23998
<910> LL/token: -8.24133
<920> LL/token: -8.24159
<930> LL/token: -8.24036
<940> LL/token: -8.2421

0	2.5	business year increase economic country budget percent economy investment service financial growth government cost sector high rate expect revenue total 
1	2.5	police officer rsipf force report vehicle operation investigation incident medium suspect public arrest station commissioner law continue involve information community 
2	2.5	service health medical provide support hospital work people care child equipment include supply receive clinic staff patient nurse centre team 
3	2.5	court case charge accuse man year allege victim prosecution matter high allegedly face public yesterday hear trial count order time 
4	2.5	government development policy national plan ensure system address sector strategy key priority stakeholder support implement consultation important implementation ministry management 
5	2.5	issue land company medium claim log concern report ques

In [13]:
from gensim.models.wrappers import LdaMallet
mallet_path = os.getcwd() + '/../mallet-2.0.8/bin/mallet'
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=texts_preprocessed,
                                                        mallet_path=mallet_path,
                                                        start=2,
                                                        limit=20,
                                                        step=1)

Mallet LDA: 2 topics, 1 topic bits, 1 topic mask
Data loaded.
max tokens: 2592
total tokens: 4283306
<10> LL/token: -8.39325
<20> LL/token: -8.26887
<30> LL/token: -8.0447
<40> LL/token: -7.9616

0	25	government country support work development project provincial national people include issue provide service year student island leader member community province 
1	25	police people year officer time public report case day week team yesterday community court follow child family man high good 

<50> LL/token: -7.92392
<60> LL/token: -7.90241
<70> LL/token: -7.88851
<80> LL/token: -7.87863
<90> LL/token: -7.87212

0	25	government country support work development project people community provincial national include provide service year island woman student issue leader important 
1	25	police people year public time officer report week case day yesterday team court follow medium member family month call election 

<100> LL/token: -7.86749
<110> LL/token: -7.8633
<120> LL/token: -7.86055
<130>

<960> LL/token: -7.83878
<970> LL/token: -7.83838
<980> LL/token: -7.83892
<990> LL/token: -7.839

0	25	government country work support people community development project island include year national provide service woman student school leader provincial important 
1	25	police people public year officer time report case week member yesterday issue day follow court team medium company add call 

<1000> LL/token: -7.83949

Total time: 1 minutes 59 seconds
Mallet LDA: 3 topics, 2 topic bits, 11 topic mask
Data loaded.
max tokens: 2592
total tokens: 4283306
<10> LL/token: -8.77387
<20> LL/token: -8.40353
<30> LL/token: -8.11343
<40> LL/token: -8.03293

0	16.66667	government country support development project work woman provide community include island national program service people important policy ensure plan training 
1	16.66667	police government public issue officer member provincial report year court case company medium parliament election statement people land yesterday man 
2	16.

<650> LL/token: -7.94207
<660> LL/token: -7.94179
<670> LL/token: -7.94065
<680> LL/token: -7.94066
<690> LL/token: -7.94133

0	16.66667	government country support development project work woman provide include national program service island ensure important community policy plan fund training 
1	16.66667	police public government officer member issue report provincial year case court medium company yesterday parliament election statement man people call 
2	16.66667	people year community student team school covid time health week day child work family good water start province area island 

<700> LL/token: -7.94076
<710> LL/token: -7.94106
<720> LL/token: -7.94041
<730> LL/token: -7.94009
<740> LL/token: -7.94064

0	16.66667	government country support development project work woman provide include national program service island ensure important community plan policy fund training 
1	16.66667	police public government officer member issue report provincial year case court medium company

<310> LL/token: -7.98428
<320> LL/token: -7.98274
<330> LL/token: -7.98168
<340> LL/token: -7.97924

0	12.5	country support community development project service provide people work include covid health island government ensure plan area water economic increase 
1	12.5	year work woman student team school good local visit training time country day child island event people start week education 
2	12.5	government issue provincial member people leader company parliament national election land province process statement add public state decision office meeting 
3	12.5	police officer case report public court people year man yesterday time charge operation rsipf accuse high follow week area force 

<350> LL/token: -7.97916
<360> LL/token: -7.97871
<370> LL/token: -7.97853
<380> LL/token: -7.97922
<390> LL/token: -7.97942

0	12.5	country support development project community service provide include work people covid health government island ensure plan area water increase economic 
1	12.5	yea

<910> LL/token: -7.98072
<920> LL/token: -7.98171
<930> LL/token: -7.98153
<940> LL/token: -7.98141

0	12.5	country support development project community provide service work include government covid health island ensure plan increase area economic improve water 
1	12.5	year woman work people student team school good country visit local training time child day island event start today education 
2	12.5	government issue provincial member people leader company parliament election land national statement process add province state public decision office year 
3	12.5	police officer case report public court people man year yesterday operation charge time rsipf accuse week area high follow force 

<950> LL/token: -7.98211
<960> LL/token: -7.98183
<970> LL/token: -7.98065
<980> LL/token: -7.98133
<990> LL/token: -7.98123

0	12.5	country support development project community provide service work include government covid health island plan ensure area increase economic improve water 
1	12.5	yea

...

<350> LL/token: -8.21301
<360> LL/token: -8.21157
<370> LL/token: -8.21089
<380> LL/token: -8.21152
<390> LL/token: -8.21204

0	2.63158	country visit island meeting support meet region official tourism regional government include security cooperation leader sign international agreement state discuss 
1	2.63158	project government development fund support infrastructure work rural plan build building funding complete ministry area road improve construction provide include 
2	2.63158	woman community training program support youth young programme child participant train work family activity awareness violence role knowledge include workshop 
3	2.63158	court case charge accuse year allege man victim matter public high prosecution allegedly face yesterday trial order hear evidence lawyer 
4	2.63158	work business year service local opportunity good country worker australian job serve islander continue time award provide offer add support 
5	2.63158	covid country case border number record test

<500> LL/token: -8.21179
<510> LL/token: -8.213
<520> LL/token: -8.21265
<530> LL/token: -8.2126
<540> LL/token: -8.21218

0	2.63158	country visit island meeting support meet official region government leader regional include security tourism cooperation sign agreement assistance discuss international 
1	2.63158	project government development fund support infrastructure work rural build plan building funding complete ministry area road construction year provide improve 
2	2.63158	woman community training program support youth young child work programme train participant activity family awareness violence important include knowledge workshop 
3	2.63158	court case charge accuse year allege man victim high matter public prosecution allegedly yesterday face order hear trial evidence lawyer 
4	2.63158	work business year service local opportunity good country australian job worker provide time continue serve islander award offer support add 
5	2.63158	covid country case border test public va

<650> LL/token: -8.21467
<660> LL/token: -8.21522
<670> LL/token: -8.2149
<680> LL/token: -8.21547
<690> LL/token: -8.21576

0	2.63158	country visit island meeting meet leader official support region government regional include security cooperation sign discuss agreement assistance state delegation 
1	2.63158	project government development fund support infrastructure work rural build plan building funding complete area ministry year road budget improve construction 
2	2.63158	woman community training program support youth young child programme work participant train activity family awareness violence knowledge important include skill 
3	2.63158	court case charge accuse year allege man victim high public matter prosecution allegedly face yesterday order hear trial represent lawyer 
4	2.63158	work business service year local opportunity country australian good provide worker job continue islander support time add award serve hard 
5	2.63158	covid country case border test public vaccine n

<800> LL/token: -8.21731
<810> LL/token: -8.2177
<820> LL/token: -8.21612
<830> LL/token: -8.21505
<840> LL/token: -8.21688

0	2.63158	country visit island meeting leader meet official support government region include regional security cooperation sign state discuss issue agreement assistance 
1	2.63158	government project development fund support work infrastructure rural build budget year building plan funding complete ministry road area construction provide 
2	2.63158	woman community training program support youth young child work programme participant train activity awareness family violence knowledge skill important people 
3	2.63158	court case charge accuse year allege man victim public matter high prosecution allegedly face yesterday hear order trial evidence lawyer 
4	2.63158	work business service local year country opportunity tourism australian good provide job continue worker support islander time add serve hard 
5	2.63158	covid country case border public test vaccine number

<950> LL/token: -8.2152
<960> LL/token: -8.21571
<970> LL/token: -8.21584
<980> LL/token: -8.2147
<990> LL/token: -8.21498

0	2.63158	country visit island meeting leader official support meet government region regional security cooperation include sign issue state agreement discuss assistance 
1	2.63158	government project development fund support work infrastructure rural budget build plan year building funding ministry complete road area construction improve 
2	2.63158	woman community training program support youth work young child programme train participant important awareness skill family activity violence knowledge role 
3	2.63158	court case charge accuse year allege man public victim high prosecution matter allegedly face yesterday hear order trial evidence represent 
4	2.63158	work business service local country opportunity tourism year provide australian worker job good islander continue time add experience support offer 
5	2.63158	covid country case border test vaccine public 

In [14]:
best_model = model_list[np.argmax(coherence_values)]
best_model.print_topics()

[(0,
  '0.030*"business" + 0.023*"country" + 0.022*"market" + 0.022*"increase" + 0.015*"year" + 0.015*"tourism" + 0.014*"percent" + 0.013*"economy" + 0.012*"economic" + 0.012*"industry"'),
 (1,
  '0.052*"government" + 0.030*"land" + 0.024*"parliament" + 0.019*"decision" + 0.018*"member" + 0.018*"issue" + 0.015*"group" + 0.015*"meeting" + 0.013*"bill" + 0.012*"minister"'),
 (2,
  '0.032*"development" + 0.026*"country" + 0.021*"policy" + 0.014*"develop" + 0.013*"region" + 0.012*"regional" + 0.012*"economic" + 0.011*"resource" + 0.010*"key" + 0.010*"plan"'),
 (3,
  '0.031*"water" + 0.022*"island" + 0.022*"area" + 0.018*"people" + 0.017*"food" + 0.017*"community" + 0.014*"disaster" + 0.011*"affect" + 0.009*"supply" + 0.008*"environment"'),
 (4,
  '0.026*"family" + 0.026*"people" + 0.019*"time" + 0.017*"home" + 0.015*"leave" + 0.015*"man" + 0.013*"day" + 0.013*"village" + 0.012*"late" + 0.012*"life"'),
 (5,
  '0.055*"woman" + 0.044*"community" + 0.032*"training" + 0.024*"program" + 0.020*"w

In [17]:
topics_df = pd.DataFrame()
for idx, val in enumerate(best_model.load_document_topics()):
    temp_df = pd.DataFrame(val)
    if topics_df.empty:
        topics_df = temp_df
    else:
        topics_df = pd.concat([topics_df, temp_df], axis=1)

KeyboardInterrupt: 

In [33]:
gensim.__version__

'4.3.2'

In [32]:
train_mlt_path = os.getcwd() + "/../mallet/mallet-2.0.8/bin/mallet"

model = gensim.models.wrappers.load(train_mlt_path)

AttributeError: module 'gensim.models.wrappers' has no attribute 'load'

In [16]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

mallet_lda_model = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(best_model)
vis_data = gensimvis.prepare(mallet_lda_model, corpus, id2word)
ldaviz_path = os.getcwd() + "/data/text/solomon_islands/st_ldavis.html"
pyLDAvis.save_html(vis_data, ldaviz_path)
pyLDAvis.display(vis_data)

/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/Users/czhang/miniforge3/envs/po/lib/python3.9/site-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


In [18]:
topics_df_new = topics_df.drop(0, axis=1).T.reset_index(drop=True)
topics_df_new["dominant_topic"] = [np.argmax(row) for _, row in topics_df_new.iterrows()]
inclu_indexs = topics_df_new[topics_df_new.dominant_topic.isin([0, 13])].index.to_list()

In [19]:
busecon_news = news.iloc[inclu_indexs].reset_index(drop=True)
busecon_news

,url,title,date,news
0,https://www.solomonstarnews.com/taro-airport-u...,"Taro airport upgrade steps up, flights to be s...",2023-08-04,\nALL Solomon Airlines flights to Taro Airport...
1,https://www.solomonstarnews.com/hcc-chc-submit...,"HCC, CHC submit multipurpose hall proposal to ...",2023-08-03,\nTHE Honiara City Council (HCC) and Central H...
2,https://www.solomonstarnews.com/in-court-with-...,In court with Assumpta,2023-08-03,\nHenderson murder trial moved to Friday\nTHE ...
3,https://www.solomonstarnews.com/bus-stabber-aw...,Bus stabber awaits sentence,2023-08-03,\nTHE young man who allegedly stole another ma...
4,https://www.solomonstarnews.com/health-scandal...,HEALTH SCANDAL AWAITS TRIAL,2023-07-11,\nFIVE people charged in connection to the $10...
...,...,...,...,...
2004,https://www.solomonstarnews.com/murder-case-ad...,Murder case adjourned,2014-04-10,AN accused charged with murder following a kil...
2005,https://www.solomonstarnews.com/where-are-the-...,Where are the solicitors?,2014-04-10,THE Public Solicitor’s Office (PSO) has been r...
2006,https://www.solomonstarnews.com/accused-jailed...,Accused jailed for life,2014-04-11,A YOUNG man in his 20’s has been convicted of ...
2007,https://www.solomonstarnews.com/accused-asks-c...,Accused asks court to grant him bail,2014-04-11,A MAN who was remanded for simple larceny has ...


In [30]:
# ss_df = pd.DataFrame()
# for news in text_lst[:-1]:
#     news_lst = news.split("All rights reserved.")
#     header = news_lst[0]
#     header_lst = header.split("\n")
#     topic = header_lst[0]
#     sample = news_lst[-1].replace("\n", " ").strip()
    
#     sid = SentimentIntensityAnalyzer()
#     ss = sid.polarity_scores(sample)
#     ss_temp = pd.DataFrame(ss, columns=ss.keys(), index=[0])
#     date_pattern = r'\b\d{1,2} (?:January|February|March|April|May|June|July|August|September|October|November|December) \d{4}\b'
#     ss_temp["date"] = re.findall(date_pattern, c[0])[0]
#     ss_temp["text"] = sample
#     ss_df = pd.concat([ss_df, ss_temp], axis=0)

ss_df = pd.DataFrame()
for news in busecon_news.news:
    sample = news.replace("\n", " ").strip()
    
    sid = SentimentIntensityAnalyzer()
    ss = sid.polarity_scores(sample)
    ss_temp = pd.DataFrame(ss, columns=ss.keys(), index=[0])
    ss_df = pd.concat([ss_df, ss_temp], axis=0)

In [31]:
busecon_news["sentiment"] = ss_df["compound"].tolist()
busecon_news["date"] = pd.to_datetime(busecon_news["date"])

In [53]:
sent_by_month = (busecon_news.set_index("date").groupby(pd.Grouper(freq="m"))[["sentiment"]].mean().reset_index())
sent_by_month['date'] = (sent_by_month['date'].dt.floor('d') + pd.offsets.MonthEnd(0) - pd.offsets.MonthBegin(1))

In [54]:
png_infl_path = os.getcwd() + "/data/prices/outputs/2017/RTFP_country_2017_2022-08-22.csv"
png_infl = pd.read_csv(png_infl_path)
png_infl["date"] = pd.to_datetime(png_infl["date"], dayfirst=True)
png_infl_sent = png_infl.merge(sent_by_month, how="left", on="date")
png_infl_sent.head(5)

,Open,High,Low,Close,Inflation,country,ISO3,date,sentiment
0,0.997417,1.035504,0.960535,0.997725,NaN,Papua New Guinea,PNG,2017-01-01,0.617313
1,0.998328,1.037044,0.960832,1.037044,NaN,Papua New Guinea,PNG,2017-02-01,0.861838
2,1.029174,1.066052,0.992296,1.012038,NaN,Papua New Guinea,PNG,2017-03-01,0.697121
3,1.019978,1.059257,0.980700,1.028941,NaN,Papua New Guinea,PNG,2017-04-01,0.889195
4,1.018736,1.061383,0.976089,1.009300,NaN,Papua New Guinea,PNG,2017-05-01,0.717284


In [90]:
sent_change = []
for i in range(len(png_infl_sent)):
    if i < 12:
        sent_change.append(np.NaN)
    else:
        a = (png_infl_sent["sentiment"][i] - png_infl_sent["sentiment"][i-12])/png_infl_sent["sentiment"][i-12]
        sent_change.append(a)

In [93]:
png_infl_sent["sent_change"] = sent_change

In [113]:
png_infl_sent["infl_diff"] = png_infl_sent["Close"].diff()
png_infl_sent["sent_diff"] = png_infl_sent["sentiment"].diff()

In [112]:
from scipy import signal
correlation = signal.correlate(c["sent_change"], c["Inflation"], mode="full")
lags = signal.correlation_lags(len(c["sent_change"]), len(c["Inflation"]), mode="full")
lag = lags[np.argmax(correlation)]
lag

27

In [118]:
import statsmodels.formula.api as smf
mod = smf.ols("infl_diff~sent_diff", data=png_infl_sent)
res = mod.fit(cov_type='HAC',cov_kwds={'maxlags':12})
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              infl_diff   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.028
Date:                Mon, 21 Aug 2023   Prob (F-statistic):              0.314
Time:                        17:56:57   Log-Likelihood:                 108.42
No. Observations:                  67   AIC:                            -212.8
Df Residuals:                      65   BIC:                            -208.4
Df Model:                           1                                         
Covariance Type:                  HAC                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0048      0.003      1.422      0.155      -0.002       0.011
sent_diff     -0.0225      0.022     -1.014      0.311      -0.066       0.021
==============================================================================
Omnibus:                        0.098   Durbin-Watson:                   2.491
Prob(Omnibus):                  0.952   Jarque-Bera (JB):                0.062
Skew:                          -0.062   Prob(JB):                        0.969
Kurtosis:                       2.917   Cond. No.                         3.69
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity and autocorrelation robust (HAC) using 12 lags and without small sample correction
"""